In [1]:
### Enviroment Setup
import os
from kaggle_secrets import UserSecretsClient

try:
    GOOGLE_API_KEY = UserSecretsClient().get_secret("GOOGLE_API_KEY")
    os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
    print("✅ Setup and authentication complete.")
except Exception as e:
    print(
        f"🔑 Authentication Error: Please make sure you have added 'GOOGLE_API_KEY' to your Kaggle secrets. Details: {e}"
    )

✅ Setup and authentication complete.


In [2]:
import logging
import os

# Clean up any previous logs
for log_file in ["logger.log", "web.log", "tunnel.log"]:
    if os.path.exists(log_file):
        os.remove(log_file)
        print(f"🧹 Cleaned up {log_file}")

# Configure logging with DEBUG log level.
logging.basicConfig(
    filename="logger.log",
    level=logging.DEBUG,
    format="%(filename)s:%(lineno)s %(levelname)s:%(message)s",
)

print("✅ Logging configured")

✅ Logging configured


In [3]:
from IPython.core.display import display, HTML
from jupyter_server.serverapp import list_running_servers


# Gets the proxied URL in the Kaggle Notebooks environment
def get_adk_proxy_url():
    PROXY_HOST = "https://kkb-production.jupyter-proxy.kaggle.net"
    ADK_PORT = "8000"

    servers = list(list_running_servers())
    if not servers:
        raise Exception("No running Jupyter servers found.")

    baseURL = servers[0]["base_url"]

    try:
        path_parts = baseURL.split("/")
        kernel = path_parts[2]
        token = path_parts[3]
    except IndexError:
        raise Exception(f"Could not parse kernel/token from base URL: {baseURL}")

    url_prefix = f"/k/{kernel}/{token}/proxy/proxy/{ADK_PORT}"
    url = f"{PROXY_HOST}{url_prefix}"

    styled_html = f"""
    <div style="padding: 15px; border: 2px solid #f0ad4e; border-radius: 8px; background-color: #fef9f0; margin: 20px 0;">
        <div style="font-family: sans-serif; margin-bottom: 12px; color: #333; font-size: 1.1em;">
            <strong>⚠️ IMPORTANT: Action Required</strong>
        </div>
        <div style="font-family: sans-serif; margin-bottom: 15px; color: #333; line-height: 1.5;">
            The ADK web UI is <strong>not running yet</strong>. You must start it in the next cell.
            <ol style="margin-top: 10px; padding-left: 20px;">
                <li style="margin-bottom: 5px;"><strong>Run the next cell</strong> (the one with <code>!adk web ...</code>) to start the ADK web UI.</li>
                <li style="margin-bottom: 5px;">Wait for that cell to show it is "Running" (it will not "complete").</li>
                <li>Once it's running, <strong>return to this button</strong> and click it to open the UI.</li>
            </ol>
            <em style="font-size: 0.9em; color: #555;">(If you click the button before running the next cell, you will get a 500 error.)</em>
        </div>
        <a href='{url}' target='_blank' style="
            display: inline-block; background-color: #1a73e8; color: white; padding: 10px 20px;
            text-decoration: none; border-radius: 25px; font-family: sans-serif; font-weight: 500;
            box-shadow: 0 2px 5px rgba(0,0,0,0.2); transition: all 0.2s ease;">
            Open ADK Web UI (after running cell below) ↗
        </a>
    </div>
    """

    display(HTML(styled_html))

    return url_prefix


print("✅ Helper functions defined.")

✅ Helper functions defined.


In [4]:
!adk create orderly-agent --model gemini-2.5-flash-lite --api_key $GOOGLE_API_KEY


Agent created in /kaggle/working/orderly-agent:
- .env
- __init__.py
- agent.py



In [21]:
%%writefile orderly-agent/agent.py

import json
from typing import Dict, Any, List

# Assuming ADK imports are structured like this for the latest version
from google.adk.agents import LlmAgent
from google.adk.tools.function_tool import FunctionTool
from google.adk.models.google_llm import Gemini
from google.genai import types
from typing import Dict, Any, List

# Session & Memory Information

from google.adk.sessions import InMemorySessionService, Session
from google.adk.memory import InMemoryMemoryService 

# --- 1. Mock Tool Implementations ---
# In a real environment, these tools would interface with the MCP (Mission Control Platform)
# or custom external APIs (e.g., databases, microservices).

def mcp_order_management_tool(order_id: str) -> str:
    """
    Retrieves the current status and details for a given order ID from the MCP system.
    Returns a JSON string containing the order status, shipment date, and items.
    """
    # Mock data simulation
    if order_id == "ORD1001":
        return json.dumps({
            "order_id": order_id,
            "status": "In Transit",
            "shipment_date": "2024-11-25",
            "estimated_delivery": "2024-11-28",
            "items": ["Galaxy S24", "Wireless Charger"],
            "tracking_number": "TRK123456"
        })
    elif order_id == "ORD1002":
        return json.dumps({
            "order_id": order_id,
            "status": "Pending Resolution",
            "issue": "Payment validation failed",
            "action_required": "Customer needs to update payment information."
        })
    else:
        return json.dumps({"order_id": order_id, "status": "Not Found", "message": "Order ID is invalid or not recognized."})

# --- 2. Specialist Agents Definition (LlmAgent) ---

order_status_agent = LlmAgent(
    name='order_status_agent',
    model='gemini-2.5-flash-lite',
    description='Creates the specialist agent responsible for handling order status queries.',
    instruction='You are the Order Status Specialist. Your primary goal is to use the mcp_order_management_tool to fetch the requested order details and present the information clearly to the customer. Do not invent information. If an order is In Transit, provide the tracking number and estimated delivery.',
    tools=[mcp_order_management_tool],
    output_key='create_order_status_agent_findings',  # The result of this agent will be stored in the session state with this key.
)

print("✅ Created Order Status Agent.")

issue_resolution_agent=LlmAgent(
        name="issue_resolution_agent",
        instruction = (
        "You are the Issue Resolution Specialist. Your task is to analyze the customer's problem "
        "and provide policy-compliant solutions. If the order status is 'Pending Resolution', "
        "use the 'mcp_order_management_tool' to understand the root cause and advise the customer "
        "on the next steps required to resolve the issue."
        ),
        model='gemini-2.5-flash-lite',
        tools=[mcp_order_management_tool],
        output_key='issue_resolution_agent_findings', 
)

print("✅ Create issue resolution agent.")

# --- 3. Main Router Agent Orchestration (LlmAgent) ---
root_agent=LlmAgent(
    instruction = 'You are the Enterprise Order Management Router Agent. Your sole responsibility is to classify the users intent and route the query to the most appropriate specialist agent provided in your tools list. If the user asks for order status or tracking, route to order_status_agent. If the user expresses a complaint, wants to cancel, or needs help fixing an issue, route to issue_resolution_agent. For general, non-order related questions, answer directly and concisely.',
    name='router_agent',
    model="gemini-2.0-flash",
    sub_agents=[order_status_agent, issue_resolution_agent],
    output_key="root_agent_findings",
)
print("✅ Router Agent Created..")

Overwriting orderly-agent/agent.py


In [22]:
url_prefix = get_adk_proxy_url()

In [ ]:
!adk web --log_level DEBUG --url_prefix {url_prefix}

/usr/local/lib/python3.11/dist-packages/google/adk/cli/fast_api.py:130: UserWarning: [EXPERIMENTAL] InMemoryCredentialService: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  credential_service = InMemoryCredentialService()
/usr/local/lib/python3.11/dist-packages/google/adk/auth/credential_service/in_memory_credential_service.py:33: UserWarning: [EXPERIMENTAL] BaseCredentialService: This feature is experimental and may change or be removed in future versions without notice. It may introduce breaking changes at any time.
  super().__init__()
INFO:     Started server process [130]
INFO:     Waiting for application startup.

+-----------------------------------------------------------------------------+
| ADK Web Server started                                                      |
|                                                                             |
| For local testing, access at http: